# PaddleOCR Testing for OCR Q&A Segmentation

This notebook tests PaddleOCR with **handwriting-optimized preprocessing**.

**Python**: 3.10 (Colab) - Compatible with PaddleOCR ✅

## Features:
- ✅ CLAHE contrast enhancement
- ✅ Bilateral filtering (edge-preserving)
- ✅ Gentle denoising for handwriting
- ✅ PaddleOCR with handwriting parameters

In [ ]:
# Clone repository
!git clone https://github.com/Abhigyan-Shekhar/ocr-qa-segmentation.git
%cd ocr-qa-segmentation

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Test OCR engines
!python test_ocr.py

In [ ]:
# Initialize OCR and Preprocessing
from src.ocr_engine import OCREngine
from src.preprocessing import ImagePreprocessor
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

print("Initializing OCR engine...")
try:
    ocr = OCREngine(engine='paddleocr')
    print("✓ Using PaddleOCR (best quality)")
except:
    ocr = OCREngine(engine='tesseract')
    print("✓ Using Tesseract (fallback)")

# Initialize preprocessor with handwriting mode
preprocessor = ImagePreprocessor(target_width=1200, enable_deskew=True)
print("✓ Preprocessor ready (handwriting mode enabled)")

print("\n🎉 System ready for handwriting recognition with enhanced preprocessing!")

## Upload Your Test Image

Upload a handwritten exam image to test OCR quality with preprocessing:

In [ ]:
from google.colab import files
import cv2
import tempfile

# Upload image
uploaded = files.upload()

if uploaded:
    # Get first uploaded file
    filename = list(uploaded.keys())[0]
    
    # Read original image
    img_original = cv2.imread(filename)
    
    # Display original
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Original Image (Before Preprocessing)')
    plt.show()
    
    print("\n" + "="*60)
    print("APPLYING HANDWRITING-OPTIMIZED PREPROCESSING...")
    print("="*60)
    print("- CLAHE contrast enhancement")
    print("- Bilateral filtering (edge-preserving)")
    print("- Gentle denoising")
    print("- Sharpening\n")
    
    # Apply preprocessing with handwriting mode
    processed_img = preprocessor.process([filename], handwriting_mode=True)
    
    # Display preprocessed
    plt.figure(figsize=(12, 8))
    plt.imshow(processed_img, cmap='gray')
    plt.axis('off')
    plt.title('After Preprocessing (Enhanced for Handwriting)')
    plt.show()
    
    # Extract text using PaddleOCR on PREPROCESSED image
    print("\n" + "="*60)
    print("RUNNING PADDLEOCR ON PREPROCESSED IMAGE...")
    print("="*60 + "\n")
    
    lines = ocr.extract_lines(processed_img)
    
    print(f"Detected {len(lines)} lines:\n")
    
    # Calculate average confidence
    avg_conf = sum(line.confidence for line in lines) / len(lines) if lines else 0
    
    for i, line in enumerate(lines[:10], 1):  # Show first 10 lines
        print(f"{i}. {line.text} (confidence: {line.confidence:.2f})")
    
    print(f"\n" + "="*60)
    print(f"AVERAGE CONFIDENCE: {avg_conf:.2f}")
    print("="*60)
    
    if avg_conf >= 0.60:
        print("✅ EXCELLENT! Preprocessing is working well!")
    elif avg_conf >= 0.40:
        print("⚠️  MODERATE: Preprocessing helped but image quality may be challenging")
    else:
        print("❌ LOW: Image quality is very poor or handwriting is extremely difficult")
        print("   Try a clearer image or check if text is actually legible")

## Compare: With vs Without Preprocessing

Run this cell to see the difference preprocessing makes:

In [ ]:
if uploaded and filename:
    print("="*60)
    print("COMPARISON: RAW vs PREPROCESSED")
    print("="*60 + "\n")
    
    # Test 1: WITHOUT preprocessing (raw image)
    print("[1] WITHOUT PREPROCESSING (Raw Image):")
    print("-" * 40)
    img_raw = cv2.imread(filename)
    # Convert to grayscale for fair comparison
    img_raw_gray = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
    lines_raw = ocr.extract_lines(img_raw_gray)
    avg_conf_raw = sum(line.confidence for line in lines_raw) / len(lines_raw) if lines_raw else 0
    
    for i, line in enumerate(lines_raw[:3], 1):
        print(f"{i}. {line.text[:50]}... (conf: {line.confidence:.2f})")
    print(f"Average Confidence: {avg_conf_raw:.2f}\n")
    
    # Test 2: WITH preprocessing
    print("[2] WITH PREPROCESSING (Enhanced):")
    print("-" * 40)
    processed_img = preprocessor.process([filename], handwriting_mode=True)
    lines_processed = ocr.extract_lines(processed_img)
    avg_conf_processed = sum(line.confidence for line in lines_processed) / len(lines_processed) if lines_processed else 0
    
    for i, line in enumerate(lines_processed[:3], 1):
        print(f"{i}. {line.text[:50]}... (conf: {line.confidence:.2f})")
    print(f"Average Confidence: {avg_conf_processed:.2f}\n")
    
    # Show improvement
    improvement = ((avg_conf_processed - avg_conf_raw) / avg_conf_raw * 100) if avg_conf_raw > 0 else 0
    
    print("="*60)
    print("RESULTS:")
    print(f"  Raw Image Confidence: {avg_conf_raw:.2f}")
    print(f"  Preprocessed Confidence: {avg_conf_processed:.2f}")
    print(f"  Improvement: {improvement:+.1f}%")
    print("="*60)